# Predicting West Nile Virus in Chicago

#### Aakash Tandel, Chaim Gluck, Matt Terribile, Nellie Carr,  August 11, 2017


### Goal:  
This project is to predict which locations in Chicago should be sprayed for mosquitos to prevent West Nile Virus based on data collected from 2007 through 2013 at 148 mosquito traps placed throughout the city of Chicago (https://www.kaggle.com/c/predict-west-nile-virus).

### Methodology and Summary:
-	Concurrently with doing exploratory data analysis (EDA) on the data, we reviewed research from scholarly and industry sources concerning factors believed to aid in the spread of WNV and more generally contribute to greater volumes of mosquitos. 
-	Based on the research we focused our feature engineering on patterns of moisture and heat over time.   For moisture, we looked at total precipitation and average wet bulb over time, theorizing that precipitation represents an increase in moisture, while wet bulb is an indicator of evaporation rate, thus decreases in moisture.   For heat, we worked with cumulative cooling degree days over time.   For each of these metrics, we created new variables for the last 7 days, the last 30 days, and the last 90 days.  For the 30 and 90 day variables, we subtracted the 7 and 30 day variables respectively for greater feature independence.  
-	We created visualizations of these vs. WNV presence over time (example below.)   Although some of the dips in the visualizations are due to the timing of the data samples, there appears to be correlation between the variables created and WNV presence.
-	While the precipitation, wet bulb, and cooling degree day features were being created, we trained and tested models using multiple algorithms to get a baseline of model performance.   Then, when we added these features, we were able to see improvements to performance.
- We then examined feature importances to do variable selection.   We achieved our best models using some of the engineered variables (30 and 90 day precipitation and cooling degree days), while removing others (wet bulb and 7 day precipitation and cooling.)

In [1]:
# Import libraries and read in data sets
import pandas as pd
import numpy as np
import csv
import matplotlib.pyplot as plt
import matplotlib.cm as cm
%matplotlib inline

import seaborn as sns
from sklearn import datasets
from sklearn import linear_model
from sklearn import metrics
from sklearn.metrics import silhouette_samples, silhouette_score

import statsmodels.api as sm
import statsmodels.formula.api as smf

import scipy.stats as stats

df_train = pd.read_csv('data_wnv/train.csv')
df_spray = pd.read_csv('data_wnv/spray.csv')
df_weather = pd.read_csv('data_wnv/weather.csv')
df_test = pd.read_csv('data_wnv/test.csv')

/Users/matthewterribile/anaconda/lib/python2.7/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


#### Exploratory Data Analysis

In [2]:
# Function to do basic EDA
def eda(dataframe):
    print "missing values \n", dataframe.isnull().sum()
    print "dataframe index \n", dataframe.index
    print "dataframe types \n", dataframe.dtypes
    print "dataframe shape \n", dataframe.shape
    print "dataframe describe \n", dataframe.describe()
    for item in dataframe:
        print item
        print dataframe[item].nunique()

eda(df_train)

missing values 
Date                      0
Address                   0
Species                   0
Block                     0
Street                    0
Trap                      0
AddressNumberAndStreet    0
Latitude                  0
Longitude                 0
AddressAccuracy           0
NumMosquitos              0
WnvPresent                0
dtype: int64
dataframe index 
RangeIndex(start=0, stop=10506, step=1)
dataframe types 
Date                       object
Address                    object
Species                    object
Block                       int64
Street                     object
Trap                       object
AddressNumberAndStreet     object
Latitude                  float64
Longitude                 float64
AddressAccuracy             int64
NumMosquitos                int64
WnvPresent                  int64
dtype: object
dataframe shape 
(10506, 12)
dataframe describe 
              Block      Latitude     Longitude  AddressAccuracy  \
count  10506.000000  1

In [3]:
df_spray.describe()

,Latitude,Longitude
count,14835.000000,14835.000000
mean,41.904828,-87.736690
std,0.104381,0.067292
min,41.713925,-88.096468
25%,41.785001,-87.794225
50%,41.940075,-87.727853
75%,41.980978,-87.694108
max,42.395983,-87.586727


In [4]:
df_weather.describe()

,Station,Tmax,Tmin,DewPoint,ResultSpeed,ResultDir
count,2944.000000,2944.000000,2944.000000,2944.000000,2944.000000,2944.000000
mean,1.500000,76.166101,57.810462,53.457880,6.960666,17.494905
std,0.500085,11.461970,10.381939,10.675181,3.587527,10.063609
min,1.000000,41.000000,29.000000,22.000000,0.100000,1.000000
25%,1.000000,69.000000,50.000000,46.000000,4.300000,7.000000
50%,1.500000,78.000000,59.000000,54.000000,6.400000,19.000000
75%,2.000000,85.000000,66.000000,62.000000,9.200000,25.000000
max,2.000000,104.000000,83.000000,75.000000,24.100000,36.000000


In [5]:
list(df_weather)

['Station',
 'Date',
 'Tmax',
 'Tmin',
 'Tavg',
 'Depart',
 'DewPoint',
 'WetBulb',
 'Heat',
 'Cool',
 'Sunrise',
 'Sunset',
 'CodeSum',
 'Depth',
 'Water1',
 'SnowFall',
 'PrecipTotal',
 'StnPressure',
 'SeaLevel',
 'ResultSpeed',
 'ResultDir',
 'AvgSpeed']

In [6]:
df_train.head(3)


,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent
0,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0
1,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0
2,2007-05-29,"6200 North Mandell Avenue, Chicago, IL 60646, USA",CULEX RESTUANS,62,N MANDELL AVE,T007,"6200 N MANDELL AVE, Chicago, IL",41.994991,-87.769279,9,1,0


In [7]:
df_train.dtypes

Date                       object
Address                    object
Species                    object
Block                       int64
Street                     object
Trap                       object
AddressNumberAndStreet     object
Latitude                  float64
Longitude                 float64
AddressAccuracy             int64
NumMosquitos                int64
WnvPresent                  int64
dtype: object

In [8]:
# Check whether 1:1 correspondance between traps and location
df_train[df_train.Trap=='T900'].Address.value_counts()

ORD Terminal 5, O'Hare International Airport, Chicago, IL 60666, USA    750
Name: Address, dtype: int64

In [9]:
# Check whether 1:1 correspondance between traps and location
print df_train[df_train.Trap=='T115'].Address.value_counts()
print df_train[df_train.Trap=='T115'].Block.value_counts()

South Doty Avenue, Chicago, IL, USA    542
Name: Address, dtype: int64
12    542
Name: Block, dtype: int64


In [10]:
# Check whether 1:1 correspondance between traps and location
print df_train[df_train.Trap=='T138'].Address.value_counts()
print df_train[df_train.Trap=='T138'].Block.value_counts()

South Stony Island Avenue, Chicago, IL, USA    314
Name: Address, dtype: int64
10    314
Name: Block, dtype: int64


In [11]:
# Check whether 1:1 correspondance between traps and location
print df_train[df_train.Trap=='T002'].Address.value_counts()
print df_train[df_train.Trap=='T002'].Block.value_counts()

4100 North Oak Park Avenue, Chicago, IL 60634, USA    185
Name: Address, dtype: int64
41    185
Name: Block, dtype: int64


##### There is a 1:1:1:1 mapping between Trap, Block,Address, and AddressNumberandStreet
For purposes of regression we only need one of them in the input data set

In [12]:
# Investigate statement: Some traps are "satellite traps". These are traps that are set up near 
# (usually within 6 blocks) an established trap to enhance surveillance efforts. 
# Satellite traps are postfixed with letters. For example, T220A is a satellite trap to T220. 
print 'Primary trap'
print df_train[df_train.Trap=='T094'].Address.value_counts()
print df_train[df_train.Trap=='T094'].Block.value_counts()
print df_train[df_train.Trap=='T094'].Latitude.value_counts()
print df_train[df_train.Trap=='T094'].Longitude.value_counts()
print 'Sattelite trap'
print df_train[df_train.Trap=='T094B'].Address.value_counts()
print df_train[df_train.Trap=='T094B'].Block.value_counts()
print df_train[df_train.Trap=='T094B'].Latitude.value_counts()
print df_train[df_train.Trap=='T094B'].Longitude.value_counts()


Primary trap
1700 West 95th Street, Chicago, IL 60643, USA    121
Name: Address, dtype: int64
17    121
Name: Block, dtype: int64
41.720848    121
Name: Latitude, dtype: int64
-87.666014    121
Name: Longitude, dtype: int64
Sattelite trap
9600 South Longwood Drive, Chicago, IL 60643, USA    5
Name: Address, dtype: int64
96    5
Name: Block, dtype: int64
41.71914    5
Name: Latitude, dtype: int64
-87.669539    5
Name: Longitude, dtype: int64


##### Satelite traps have different block, latitude, and longitude data than primary traps

In [13]:
### Matt started here
# change type of date
# Not sure what address accuracy is - values 8,9,5,3
# No null values
# Difference between mosquito types? - make dummy variables for mosquito species
# Traps all have at least one mosquito
# 10506 rows, 12 columns
# Address number and street are redundant of address, perhaps of latitude / longitude?
# 136 traps.   Do they stay at the same place?

In [14]:
#Date                       object  - convert to date format
#Address                    object  - redundant to Trap, remove for regression, may want zip code from
#Species                    object  - 7 types, convert to dummy variable?  Or is it useful in regression?
#                                          - some more likely to carry virus?
#Block                       int64  - redundant to Trap, remove for regression
#Street                     object  - redundant to Trap, remove for regression
#Trap                       object  - do we want to regress on this as categorical or  Lat&Long? 
#AddressNumberAndStreet     object  - redundant to Trap, remove for regression
#Latitude                  float64  - do we want to regress on this?   Don't think we want to treat as continuous variable
#Longitude                 float64  - do we want to regress on this?   Don't think we want to treat as continuous variable
#AddressAccuracy             int64  - 3,5,9,8 from GeoCoder.   See defn below - This probably doesn't belong in the regression
#                                    - When a 3 or 5 has the virus, may want to spray wider area
#NumMosquitos                int64  - 1 to 50 - when above 50, another record is created
#                                    - Not sure how to treat multiple rows for same trap and same day
#                                    - Some multiples are different mosquito, some are the same, but may be Y/N on WNV
#WnvPresent                  int64  - 0 (WNV not present, about 95%) or 1 (WNV present, about 5%)



In [15]:
# Understand what address accuracy field is
# address accuracy counts
df_train.AddressAccuracy.value_counts()

8    4628
9    3980
5    1807
3      91
Name: AddressAccuracy, dtype: int64

##### Definition of Address Accuracy
Address Accuracy - When the geocoder receives a query it attempts to find the best matched feature for the address 
given. Depending on the amount of data we have for the area concerned, it may be able to identify a 
specific building that matches, or it may only be able to match the street or district.  
If it can only match a higher level feature such as the district it will still return a lat/long,
but it will be for the centroid of that feature, which is likely to be offset from the location 
of the specific address requested. The Accuracy tells you the scale of the object that matched, 
so you know how precise the geocode is likely to be.
https://www.experts-exchange.com/questions/27986524/How-do-I-translate-the-Google-maps-v3-Geocode-location-type-to-a-Geocode-score-0-9.html
3 = Sub-region (county, municipality, etc.) level accuracy  - 91 of these
5 = Post code (zip code) level accuracy  - 1807 of these
8 = Address level accuracy  -  4628 of these - this is very accurate
9 = Premise (building name, property name, shopping center, etc.) level accuracy - 3980 of these - this is very accurate

For our analysis, this is likely not a key variable

In [16]:
# Need to understand how many records are from the same date with the same trap.   Should we combine them?

# Create a dataframe with just date, trap number, and species.   Check for how many duplicates
df_temp = df_train[['Date','Trap','Species']]
df_temp.duplicated().sum()

2031

###### Many traps have multiple records for the same day.   If more than 50 of a particular mosquito, a new record is made.

##### Check if all mosquito types have WNV

In [17]:
df_train[df_train.WnvPresent==1].Species.value_counts()


CULEX PIPIENS/RESTUANS    262
CULEX PIPIENS             240
CULEX RESTUANS             49
Name: Species, dtype: int64

In [18]:
# compare to those without Wnv
df_train[df_train.WnvPresent==0].Species.value_counts()

CULEX PIPIENS/RESTUANS    4490
CULEX RESTUANS            2691
CULEX PIPIENS             2459
CULEX TERRITANS            222
CULEX SALINARIUS            86
CULEX TARSALIS               6
CULEX ERRATICUS              1
Name: Species, dtype: int64

In [19]:
'''  Quick calculation in xls.   Some type of mosquitos more likely to have WNV.
Rows			                     No WNV	WNV	Total	% with
CULEX PIPIENS/RESTUANS    			4490	262	4752	5.5%
CULEX RESTUANS           			2691	49	2740	1.8%
CULEX PIPIENS             			2459	240	2699	8.9%
CULEX TERRITANS            			222	    0	222	    0.0%
CULEX SALINARIUS        			86	    0	86	    0.0%
CULEX TARSALIS          			6	    0	6	    0.0%
CULEX ERRATICUS            			1	    0	1	    0.0%
			                        9955	551	10506	5.2%'''

'  Quick calculation in xls.   Some type of mosquitos more likely to have WNV.\nRows\t\t\t                     No WNV\tWNV\tTotal\t% with\nCULEX PIPIENS/RESTUANS    \t\t\t4490\t262\t4752\t5.5%\nCULEX RESTUANS           \t\t\t2691\t49\t2740\t1.8%\nCULEX PIPIENS             \t\t\t2459\t240\t2699\t8.9%\nCULEX TERRITANS            \t\t\t222\t    0\t222\t    0.0%\nCULEX SALINARIUS        \t\t\t86\t    0\t86\t    0.0%\nCULEX TARSALIS          \t\t\t6\t    0\t6\t    0.0%\nCULEX ERRATICUS            \t\t\t1\t    0\t1\t    0.0%\n\t\t\t                        9955\t551\t10506\t5.2%'

##### Check if % or mosquitos with WNV is statistically different by mosquito type

In [20]:
# Do chi squared test
from scipy import stats
import numpy as np
# do test for  Restuans vs. Pipiens / Restuans
disease = np.array([[4752,262], [2740,49]])
stats.chi2_contingency(disease)

(55.436573984211044,
 9.652371892086369e-14,
 1,
 array([[ 4814.15968217,   199.84031783],
        [ 2677.84031783,   111.15968217]]))

In [21]:
# output is x-squared and p value
# p-value is small so unlikely from same sample

# try again with Pipiens vs. Pipiens / Restuans
disease = np.array([[4752,262], [2699,240]])
stats.chi2_contingency(disease)

(26.600088228381278,
 2.5023232591543785e-07,
 1,
 array([[ 4697.51213379,   316.48786621],
        [ 2753.48786621,   185.51213379]]))

In [22]:
# p-value is small so unlikely from same sample

##### WNV is more prevalent in some species, but we still need to consider all species

#### Change formating of some columns

In [23]:
## Change date column to datetime and add a Year and Day of Year column
df_train['Date']= pd.to_datetime(df_train.Date)
df_train['Year']= df_train.Date.dt.year
df_train['DayofYear']= df_train.Date.dt.dayofyear

In [24]:
df_train.dtypes

Date                      datetime64[ns]
Address                           object
Species                           object
Block                              int64
Street                            object
Trap                              object
AddressNumberAndStreet            object
Latitude                         float64
Longitude                        float64
AddressAccuracy                    int64
NumMosquitos                       int64
WnvPresent                         int64
Year                               int64
DayofYear                          int64
dtype: object

In [25]:
# Label encode the trap column
from sklearn import ensemble, preprocessing
lbls = preprocessing.LabelEncoder()
lbls.fit(list(df_train['Trap'].values) + list(df_test['Trap'].values))
df_train['Trap'] = lbls.transform(df_train['Trap'].values)
df_test['Trap'] = lbls.transform(df_test['Trap'].values)

In [26]:
# Label encode the Species column
lbls.fit(list(df_train['Species'].values) + list(df_test['Species'].values))
df_train['Species'] = lbls.transform(df_train['Species'].values)
df_test['Species'] = lbls.transform(df_test['Species'].values)

In [27]:
# Sort train dataframe by date,trap, and species to group like rows together

df_train.sort_values(['Date', 'Trap','Species'], ascending=[True, True,True],inplace=True)

In [28]:
# Build a new data frame that aggregates the rows with the same date, trap, and species number

# Start by placing first row of existing dataframe in new dataframe
df_train2 = df_train[0:1]

# Get column indices for number of mosquitos and West Nile Virus present.   
Mosq_col = df_train.columns.get_loc("NumMosquitos")
Wnv = df_train.columns.get_loc("WnvPresent")

# Loop through the existing data frame starting with the second row.   
for i in range(1,len(df_train)):

    # If the date, trap number, and species are the same as the last row in the new dataframe:
    if ((df_train.iloc[i].Date == df_train2.iloc[-1].Date)&(df_train.iloc[i].Trap == df_train2.iloc[-1].Trap)&(df_train.iloc[i].
                                                                                                                               Species== df_train2.iloc[-1].Species)):
    #   add the mosquitos numbers on to the current values in last row of new dataframe
         df_train2.set_value(-1,Mosq_col, (df_train2.iloc[-1].NumMosquitos + df_train.iloc[i].NumMosquitos),takeable=True )

        #   if WNV = 1 then change WNV to 1
         if (df_train.iloc[i].WnvPresent == 1):
            df_train2.set_value(-1,Wnv,1,takeable=True )

    else:
    #   add the row to the end of the dataframe
        df_train2 = df_train2.append(df_train[i:i+1])
   

In [29]:
df_train2.shape

(8475, 14)

In [30]:
# drop columns not using in initial regressions
drop_cols = ['Address','Block','Street','AddressNumberAndStreet','AddressAccuracy']

df_train3 = df_train2.drop(drop_cols,axis=1)



In [31]:
df_train3.shape

(8475, 9)

In [32]:
df_train3.head(10)

,Date,Species,Trap,Latitude,Longitude,NumMosquitos,WnvPresent,Year,DayofYear
0,2007-05-29,2,1,41.954690,-87.800991,1,0,2007,149
1,2007-05-29,3,1,41.954690,-87.800991,1,0,2007,149
2,2007-05-29,3,8,41.994991,-87.769279,1,0,2007,149
3,2007-05-29,2,15,41.974089,-87.824812,1,0,2007,149
4,2007-05-29,3,15,41.974089,-87.824812,4,0,2007,149
5,2007-05-29,3,34,41.921600,-87.666455,2,0,2007,149
6,2007-05-29,3,35,41.891118,-87.654491,1,0,2007,149
7,2007-05-29,2,37,41.867108,-87.654224,1,0,2007,149
8,2007-05-29,3,37,41.867108,-87.654224,2,0,2007,149
9,2007-05-29,3,38,41.896282,-87.655232,1,0,2007,149


In [33]:
# df_train3.to_csv('df_train3.csv',index=False)


In [34]:
df_train4 = pd.read_csv('df_train3.csv')

In [35]:
df_train4.head(6)

,Trap,NumMosquitos,WnvPresent,Year,DayofYear,CULEX ERRATICUS,CULEX PIPIENS,CULEX PIPIENS/RESTUANS,CULEX RESTUANS,CULEX SALINARIUS,CULEX TARSALIS,CULEX TERRITANS
0,T002,2,0,2007,149,0,0,1,1,0,0,0
1,T007,1,0,2007,149,0,0,0,1,0,0,0
2,T015,5,0,2007,149,0,0,1,1,0,0,0
3,T045,2,0,2007,149,0,0,0,1,0,0,0
4,T046,1,0,2007,149,0,0,0,1,0,0,0
5,T048,3,0,2007,149,0,0,1,1,0,0,0


#### Read in cleaned weather data and engineer additional features

In [36]:
# Read in weather data 
weather_stn = pd.read_csv('cleaned_weather.csv')

In [37]:
# Convert date to to date_time format
weather_stn['date']= pd.to_datetime(weather_stn.date)

In [38]:
# Check fields
weather_stn.dtypes

date             datetime64[ns]
tmax_x                    int64
tmin_x                    int64
tavg_x                  float64
depart_x                float64
dewpoint_x                int64
wetbulb_x               float64
heat_x                  float64
cool_x                  float64
preciptotal_x           float64
stnpressure_x           float64
sealevel_x              float64
resultspeed_x           float64
resultdir_x               int64
avgspeed_x              float64
tmax_y                    int64
tmin_y                    int64
tavg_y                  float64
depart_y                float64
dewpoint_y                int64
wetbulb_y               float64
heat_y                  float64
cool_y                  float64
preciptotal_y           float64
stnpressure_y           float64
sealevel_y              float64
resultspeed_y           float64
resultdir_y               int64
avgspeed_y              float64
dtype: object

In [39]:
# Sort by date
weather_stn2 = weather_stn.sort_values(['date'], ascending=[True])

In [40]:
# Add a year column to be sure not crossing year boundaries since data only 6 months / yr
weather_stn2['year']= weather_stn2.date.dt.year

In [41]:
# Create new features, total cooling days in last 7 days, last 30 minus 7 days,and last 90 minus 30 days

# fcn to pull the data in a particular column of a dataframe from days_ago prior to the index
def pull_val(df,col_name,index,days_ago):
    if ((df['year'].iloc[i])== (df['year'].iloc[i-days_ago])):   # check that years match
        try:                                        # need 'try' for start of the data frame
            val = df[col_name].iloc[i-days_ago]   # find value from number of days_ago
        except:
            val = 0         # if prior to start of dataframe, assume 0
    else:                     # if years don't match, assume 0
        val = 0
    return val

In [42]:
# Add feature for last 7 days cooling

# Initialize the column for total cooling days in last 7 days
weather_stn2['coollast7'] = weather_stn2['cool_x']
col_index = weather_stn2.columns.get_loc('coollast7')    # get column index corresponding to column name

# loop through all the rows of the condensed weather data frame

for i in range(len(weather_stn2)):
    coollast7 = 0      #initialize value to current day
    for j in range(0,7):       # add the last 7 days' values
        coollast7 += pull_val(weather_stn2,'cool_x',i,j)
         # add the value to the dataframe
        weather_stn2.set_value(i,col_index,coollast7,takeable=True )   #takeable = True uses indices


In [43]:
# Add feature for last 30 days cooling.   Later will subtract last 7 days from it.

# Initialize the column for total cooling days in last 30 days
weather_stn2['coollast30'] = weather_stn2['cool_x']
col_index = weather_stn2.columns.get_loc('coollast30')    # get column index corresponding to column name

# loop through all the rows of the condensed weather data frame

for i in range(len(weather_stn2)):
    coollast30 = 0      #initialize value to current day
    for j in range(0,30):       # add the last 30 days' values
        coollast30 += pull_val(weather_stn2,'cool_x',i,j)
    # add the value to the dataframe
    weather_stn2.set_value(i,col_index,coollast30,takeable=True )   #takeable = True uses indices


In [44]:
# Add feature for last 90 days cooling.   Later will subtract last 30 days from it.

# Initialize the column for total cooling days in last 90 days
weather_stn2['coollast90'] = weather_stn2['cool_x']
col_index = weather_stn2.columns.get_loc('coollast90')    # get column index corresponding to column name

# loop through all the rows of the condensed weather data frame

for i in range(len(weather_stn2)):
    coollast90 = 0      #initialize value to current day
    for j in range(0,90):       # add the last 90 days' values
        coollast90 += pull_val(weather_stn2,'cool_x',i,j)
    # add the value to the dataframe
    weather_stn2.set_value(i,col_index,coollast90,takeable=True )   #takeable = True uses indices


In [45]:
# Add feature for precipitation for last 7, 30, 90 days

In [46]:
# Add feature for last 7 days precipitation.

# Initialize the column for total precipitation in last 7 days
weather_stn2['preciplast7'] = weather_stn2['preciptotal_x']
col_index = weather_stn2.columns.get_loc('preciplast7')    # get column index corresponding to column name

# loop through all the rows of the condensed weather data frame

for i in range(len(weather_stn2)):
    preciplast7 = 0      #initialize value to current day
    for j in range(0,7):       # add the last 7 days' values
        preciplast7 += pull_val(weather_stn2,'preciptotal_x',i,j)
         # add the value to the dataframe
        weather_stn2.set_value(i,col_index,preciplast7,takeable=True )   #takeable = True uses indices


In [47]:
# Add feature for last 30 days precipitation.   Later will subtract last 7 days from it.

# Initialize the column for total precipitation in last 30 days
weather_stn2['preciplast30'] = weather_stn2['preciptotal_x']
col_index = weather_stn2.columns.get_loc('preciplast30')    # get column index corresponding to column name

# loop through all the rows of the condensed weather data frame

for i in range(len(weather_stn2)):
    preciplast30 = 0      #initialize value to current day
    for j in range(0,30):       # add the last 30 days' values
        preciplast30 += pull_val(weather_stn2,'preciptotal_x',i,j)
         # add the value to the dataframe
        weather_stn2.set_value(i,col_index,preciplast30,takeable=True )   #takeable = True uses indices


In [48]:
# Add feature for last 90 days precipitation.   Later will subtract last 30 days from it.

# Initialize the column for total precipitation in last 90 days
weather_stn2['preciplast90'] = weather_stn2['preciptotal_x']
col_index = weather_stn2.columns.get_loc('preciplast90')    # get column index corresponding to column name

# loop through all the rows of the condensed weather data frame

for i in range(len(weather_stn2)):
    preciplast90 = 0      #initialize value to current day
    for j in range(0,90):       # add the last 90 days' values
        preciplast90 += pull_val(weather_stn2,'preciptotal_x',i,j)
         # add the value to the dataframe
        weather_stn2.set_value(i,col_index,preciplast90,takeable=True )   #takeable = True uses indices


In [49]:
# Add feature for wet bulb for last 7, 30, 90 days

In [50]:
# Add feature for last 7 days wetbulb

# Initialize the column for total wet bulb in last 7 days
weather_stn2['wblast7'] = (weather_stn2['wetbulb_x']+weather_stn2['wetbulb_y'])/2 # take average of 2 stns
col_index = weather_stn2.columns.get_loc('wblast7')    # get column index corresponding to column name

# loop through all the rows of the condensed weather data frame

for i in range(len(weather_stn2)):
    wblast7 = 0      #initialize value to current day
    for j in range(0,7):       # add the last 7 days' values
        wblast7 += (pull_val(weather_stn2,'wetbulb_x',i,j)+pull_val(weather_stn2,'wetbulb_y',i,j))/2
         # add the value to the dataframe
        weather_stn2.set_value(i,col_index,wblast7,takeable=True )   #takeable = True uses indices


In [51]:
# Add feature for last 30 days wet bulb

# Initialize the column for total wet bulb in last 30 days
weather_stn2['wblast30'] = (weather_stn2['wetbulb_x']+weather_stn2['wetbulb_y'])/2 # take average of 2 stns
col_index = weather_stn2.columns.get_loc('wblast30')    # get column index corresponding to column name

# loop through all the rows of the condensed weather data frame

for i in range(len(weather_stn2)):
    wblast30 = 0      #initialize value to current day
    for j in range(0,30):       # add the last 7 days' values
        wblast30 += (pull_val(weather_stn2,'wetbulb_x',i,j)+pull_val(weather_stn2,'wetbulb_y',i,j))/2
         # add the value to the dataframe
        weather_stn2.set_value(i,col_index,wblast30,takeable=True )   #takeable = True uses indices


In [52]:
# Add feature for last 90 days wetbulb

# Initialize the column for total wet bulb in last 90 days
weather_stn2['wblast90'] = (weather_stn2['wetbulb_x']+weather_stn2['wetbulb_y'])/2 # take average of 2 stns
col_index = weather_stn2.columns.get_loc('wblast90')    # get column index corresponding to column name

# loop through all the rows of the condensed weather data frame

for i in range(len(weather_stn2)):
    wblast90 = 0      #initialize value to current day
    for j in range(0,90):       # add the last 7 days' values
        wblast90 += (pull_val(weather_stn2,'wetbulb_x',i,j)+pull_val(weather_stn2,'wetbulb_y',i,j))/2
         # add the value to the dataframe
        weather_stn2.set_value(i,col_index,wblast90,takeable=True )   #takeable = True uses indices


In [53]:
# Net out the last 7 days from last 30 and last 30 from last 90 for cooling days, precip, and wb

weather_stn2['preciplast90'] = weather_stn2['preciplast90'] - weather_stn2['preciplast30']
weather_stn2['preciplast30'] = weather_stn2['preciplast30'] - weather_stn2['preciplast7']

weather_stn2['coollast90'] = weather_stn2['coollast90'] - weather_stn2['coollast30']
weather_stn2['coollast30'] = weather_stn2['coollast30'] - weather_stn2['coollast7']

weather_stn2['wblast90'] = weather_stn2['wblast90'] - weather_stn2['wblast30']
weather_stn2['wblast30'] = weather_stn2['wblast30'] - weather_stn2['wblast7']

In [54]:
#write file to csv
#weather_stn2.to_csv('weather_stn2.csv',index=False)

#### Merge weather data and training data

In [55]:
# Merge updated weather file with train file
train_weather2 = pd.merge(df_train3, weather_stn2, left_on='Date', right_on='date',how='left')

In [56]:
train_weather2.dtypes

Date             datetime64[ns]
Species                   int64
Trap                      int64
Latitude                float64
Longitude               float64
NumMosquitos              int64
WnvPresent                int64
Year                      int64
DayofYear                 int64
date             datetime64[ns]
tmax_x                    int64
tmin_x                    int64
tavg_x                  float64
depart_x                float64
dewpoint_x                int64
wetbulb_x               float64
heat_x                  float64
cool_x                  float64
preciptotal_x           float64
stnpressure_x           float64
sealevel_x              float64
resultspeed_x           float64
resultdir_x               int64
avgspeed_x              float64
tmax_y                    int64
tmin_y                    int64
tavg_y                  float64
depart_y                float64
dewpoint_y                int64
wetbulb_y               float64
heat_y                  float64
cool_y  

In [57]:
train_weather2.columns

Index([u'Date', u'Species', u'Trap', u'Latitude', u'Longitude',
       u'NumMosquitos', u'WnvPresent', u'Year', u'DayofYear', u'date',
       u'tmax_x', u'tmin_x', u'tavg_x', u'depart_x', u'dewpoint_x',
       u'wetbulb_x', u'heat_x', u'cool_x', u'preciptotal_x', u'stnpressure_x',
       u'sealevel_x', u'resultspeed_x', u'resultdir_x', u'avgspeed_x',
       u'tmax_y', u'tmin_y', u'tavg_y', u'depart_y', u'dewpoint_y',
       u'wetbulb_y', u'heat_y', u'cool_y', u'preciptotal_y', u'stnpressure_y',
       u'sealevel_y', u'resultspeed_y', u'resultdir_y', u'avgspeed_y', u'year',
       u'coollast7', u'coollast30', u'coollast90', u'preciplast7',
       u'preciplast30', u'preciplast90', u'wblast7', u'wblast30', u'wblast90'],
      dtype='object')

In [58]:
# Drop 2nd date column
train_weather2.drop('date', axis=1,inplace=True)
train_weather2.dtypes

Date             datetime64[ns]
Species                   int64
Trap                      int64
Latitude                float64
Longitude               float64
NumMosquitos              int64
WnvPresent                int64
Year                      int64
DayofYear                 int64
tmax_x                    int64
tmin_x                    int64
tavg_x                  float64
depart_x                float64
dewpoint_x                int64
wetbulb_x               float64
heat_x                  float64
cool_x                  float64
preciptotal_x           float64
stnpressure_x           float64
sealevel_x              float64
resultspeed_x           float64
resultdir_x               int64
avgspeed_x              float64
tmax_y                    int64
tmin_y                    int64
tavg_y                  float64
depart_y                float64
dewpoint_y                int64
wetbulb_y               float64
heat_y                  float64
cool_y                  float64
precipto

In [59]:
# Write training file with weather data to csv
train_weather2.to_csv('train_weather2e.csv',index=False)

##### Format test file similar to train file

In [60]:
df_test.columns

Index([u'Id', u'Date', u'Address', u'Species', u'Block', u'Street', u'Trap',
       u'AddressNumberAndStreet', u'Latitude', u'Longitude',
       u'AddressAccuracy'],
      dtype='object')

In [61]:
df_test.dtypes

Id                          int64
Date                       object
Address                    object
Species                     int64
Block                       int64
Street                     object
Trap                        int64
AddressNumberAndStreet     object
Latitude                  float64
Longitude                 float64
AddressAccuracy             int64
dtype: object

In [64]:
## Change date column type
df_test['Date']= pd.to_datetime(df_test.Date)

In [65]:
# Drop columns not using in initial regressions
drop_cols = ['Address','Block','Street','AddressNumberAndStreet','AddressAccuracy']
df_test3 = df_test.drop(drop_cols,axis=1)

In [66]:
df_test3.columns

Index([u'Id', u'Date', u'Species', u'Trap', u'Latitude', u'Longitude'], dtype='object')

In [67]:
# Merge test data with weather data, including added features

In [68]:
test_weather3 = pd.merge(df_test3, weather_stn2, left_on='Date', right_on='date',how='left')

In [69]:
test_weather3.columns

Index([u'Id', u'Date', u'Species', u'Trap', u'Latitude', u'Longitude', u'date',
       u'tmax_x', u'tmin_x', u'tavg_x', u'depart_x', u'dewpoint_x',
       u'wetbulb_x', u'heat_x', u'cool_x', u'preciptotal_x', u'stnpressure_x',
       u'sealevel_x', u'resultspeed_x', u'resultdir_x', u'avgspeed_x',
       u'tmax_y', u'tmin_y', u'tavg_y', u'depart_y', u'dewpoint_y',
       u'wetbulb_y', u'heat_y', u'cool_y', u'preciptotal_y', u'stnpressure_y',
       u'sealevel_y', u'resultspeed_y', u'resultdir_y', u'avgspeed_y', u'year',
       u'coollast7', u'coollast30', u'coollast90', u'preciplast7',
       u'preciplast30', u'preciplast90', u'wblast7', u'wblast30', u'wblast90'],
      dtype='object')

In [70]:
# add day of year
test_weather3['DayofYear']= test_weather3.Date.dt.dayofyear

# drop extra date column
test_weather3 = test_weather3.drop(['date'],axis=1)

# sort test file by 'Id' for Kaggle
test_weather3.sort_values(['Id'], ascending=[True],inplace=True)


In [71]:
test_weather3.isnull().sum()

Id               0
Date             0
Species          0
Trap             0
Latitude         0
Longitude        0
tmax_x           0
tmin_x           0
tavg_x           0
depart_x         0
dewpoint_x       0
wetbulb_x        0
heat_x           0
cool_x           0
preciptotal_x    0
stnpressure_x    0
sealevel_x       0
resultspeed_x    0
resultdir_x      0
avgspeed_x       0
tmax_y           0
tmin_y           0
tavg_y           0
depart_y         0
dewpoint_y       0
wetbulb_y        0
heat_y           0
cool_y           0
preciptotal_y    0
stnpressure_y    0
sealevel_y       0
resultspeed_y    0
resultdir_y      0
avgspeed_y       0
year             0
coollast7        0
coollast30       0
coollast90       0
preciplast7      0
preciplast30     0
preciplast90     0
wblast7          0
wblast30         0
wblast90         0
DayofYear        0
dtype: int64

In [72]:
# Write test file combined with weather to csv
test_weather3.to_csv('test_weather3e.csv',header=True,index=False)

#### Data was now fed to XGBoost and then to Kaggle